<style>
@import url(https://www.numfys.net/static/css/nbstyle.css);
</style>
<a href="https://www.numfys.net"><img class="logo" /></a>

# Eigenenergies of the Double-Well Potential
## Examples - Quantum Mechanics
<section class="post-meta">
By Henning G. Hugdal, Magnus H-S Dahle and Peter Berg
</section>
Las edited: January 19th 2019
___

Потенциалы с двумя минимумами часто называют двухъямными потенциалами. Они имеют широкий спектр применений в физике в целом, включая квантовую механику. Двумя примерами являются:

- Водородная связь: например, протон (ион водорода) между атомами кислорода двух молекул воды будет находиться в двухъямном потенциале. Следовательно, он будет стремиться быть ближе к одному из двух атомов.

- Инверсия аммиака: В аммиаке, $NH_3$, атом азота расположен над плоскостью, образованной тремя атомами водорода. Атом азота может протолкнуться через плоскость водорода, так что теперь у вас есть противоположно направленная, но в остальном идентичная молекула. Это может соответствовать определенному двухъямному потенциалу, как показано на рисунке выше. Вы можете прочитать больше об инверсии аммиака в [Лекциях Фейнмана по физике](http://www.feynmanlectures.caltech.edu/III_08.html#Ch8-S6 "The Feynman Lectures on Physics, Vol III").

<img src="images/6qm_ammonia_inversion.png" style="height:400px">

Целью этого примера является вычисление собственных энергий низших состояний для одномерного двухъямного потенциала

$$
\qquad V(x) = \frac{V_\mathrm{max}}{b^4}(x^2-b^2)^2,
$$

где $V_\mathrm{max}$ и $b$ являются константами. Мы будем использовать ту же итеративную формулу, что и полученную в [этом примере](https://nbviewer.jupyter.org/urls/www.numfys.net/media/notebooks/ex_qm1_numerical_determination_of_eigenvalues_for_harmonic_oscillator.ipynb), который вычисляет волновую функцию в точке $x_{i+1}$ на основе ее значения в двух предыдущих точках, согласно

$$
\qquad \psi_{i+1} = -\psi_{i-1} + 2\left[1 - (\Delta x)^2 \left(E-V(x_i)\right)\right] \psi_i,
$$

где мы установили $\hbar=m=1$ для простоты. Первая итерация должна обрабатываться отдельно; используя начальные значения $\psi_0$ и $\psi_0'$, $\psi_1$ можно рассчитать в наименьшем порядке из

$$
\qquad \psi_1 = \psi_0 + \Delta x \cdot \psi_0'.
$$

Это затем позволяет нам вычислить $\psi_i$ для более высоких значений $i$. Поскольку потенциал симметричен относительно $x=0$, мы знаем, что волновые функции должны быть либо симметричными, либо антисимметричными, при этом возбужденное состояние $n$ имеет четность $(-1)^n$ и $n$ узлов. Это дает всю информацию, необходимую нам для расчета собственных энергий.

Мы найдем собственные энергии основного и первого возбужденного состояний для двух разных значений $V_\mathrm{max}$, чтобы увидеть, как потенциальный барьер влияет на систему. Мы выбираем значение $b=0,2$ на протяжении всего примера.

Во-первых, мы импортируем необходимые модули. Затем мы определяем некоторые функции, которые будут использоваться для нахождения собственных энергий и построения волновых функций:

- Результаты: кортеж, используемый для возврата результатов вычислений.
- forwardShoot(): функция, которая используется для вычисления собственной энергии и волновой функции для заданного потенциала.
- normPlot(): нормализует и строит волновую функцию.

Это избавляет нас от написания большого количества кода, так как одна и та же процедура повторяется для основного и первого возбужденного состояния для двух разных значений $V_\mathrm{max}$. Обратите внимание, что функция forwardShoot() работает только для основного и первого возбужденного состояний. (На самом деле, она работает для всех состояний, в которых волновая функция $\psi_n$ приближается к 0 сверху для больших значений $x$, т. е. $\lim_{x \to \infty} \psi_n(x) \rightarrow 0^{+}$. Это относится к $\psi_n$ с $n \in \{0,1,4,5,8,9,\dots\}$.) Чтобы рассматривать второе и третье возбужденные состояния, необходимо было бы изменить условия, используемые для метода деления пополам.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from collections import namedtuple

# Set common figure parameters:
newparams = {'axes.labelsize': 14, 'axes.linewidth': 1, 'savefig.dpi': 300, 
             'lines.linewidth': 1.0, 'figure.figsize': (8, 3),
             'figure.subplot.wspace': 0.4, 'figure.dpi': 200,
             'ytick.labelsize': 10, 'xtick.labelsize': 10,
             'ytick.major.pad': 5, 'xtick.major.pad': 5,
             'legend.fontsize': 10, 'legend.frameon': False, 
             'legend.handlelength': 1.5}
plt.rcParams.update(newparams)

In [ ]:
Results = namedtuple('res', ['func','E'])

def forwardShoot(V, dx, psi_0, dpsi_0, E_l=0.0, E_u=400.0, acc=1e-10):
    """Takes a potential and a vector of zeroes and uses the forward shooting method to calculate the eigenenergy
    using the upper and lower bounds given.
    
    Arguments:
        V       Potential
        dx      Step length x-axis
        psi_0   Initial value of wavefunction at x=0
        dpsi_0  Initial value of first derivative of wavefunction at x=0
        E_l     Lower bound of energy
        E_u     Upper bound of energy
        acc     Accuracy when determining the energy
        
    Returns tuple with elements
        func    Wavefunction
        E       Eigenenergy
    """
    
    f = np.zeros(len(V))
    f[0] = psi_0
    df_0 = dpsi_0
    
    E1 = E_l  # Lower bound for eigenenergy
    E2 = E_u  # Upper bound for eigenenergy
    E = E1
    DeltaE = 1
    while DeltaE > acc:  
        for i, V_ in enumerate(V[0:-1]):
            if i==0:
                f[i+1]=f[i]+dx*df_0
            else:
                f[i+1] = -f[i-1]+2*f[i]*(1-dx**2*(E-V_))
            # Implementation of bisection method. If the function value is out of bounds,
            # a new value for the energy is chosen. When the difference between the upper
            # and lower bound for the energy is smaller than the given accuracy,
            # the while loop stops, yielding a result for E.
            if f[i] > 20:
                E1 = E
                E = E1 + (E2-E1)/2
                break
            elif f[i] <- 20:
                E2 = E
                E = E2 - (E2-E1)/2
                break
        DeltaE = E2-E1
    res = Results(f, E)
    return res

def normPlot(V, func, E, x, x_end, title='', sym=True, legend_loc=1):
    """Normalizes and plots the wavefunction together with the potential and eigenenergy.
    
    Arguments:
        V            Potential
        func         Wavefunction
        E            Eigenenergy
        x            Vector with x-values
        x_end        Biggest x-value to be included in normalization/plot
        title        Plot title
        sym          True if symmetric wavefunction, false if anti-symmetric
        legend_loc   Placement of plot legend
        
    Returns:
        F            Normalized wavefunction.
    """
    
    dx = x[1]-x[0]
    sign = -1
    if sym: sign = 1
    c = sum(2*func[0:int(x_end/dx)]**2*dx)
    F = func/np.sqrt(c)
    
    plt.figure()
    p1, = plt.plot(x, V/V[0], 'b')
    p2, = plt.plot(x, F, 'g')
    p3, = plt.plot([-x_end,x_end], [E/V[0], E/V[0]], 'r--')
    plt.legend([p1,p2,p3], [r'Potential $V(x)/V_\mathrm{max}$',r'Wave function $\psi(x)$',
                            r'Eigenenergy $E/V_\mathrm{max}$'], loc=legend_loc)
    plt.plot(-x, V/V[0], 'b', -x, sign*F, 'g', [-x_end,x_end], [0,0], 'k:')
    plt.ylim([min(sign*F[0:int(x_end/dx)])-0.1, max(F[0:int(x_end/dx)])+0.5])
    plt.xlim([-x_end, x_end])
    plt.xlabel(r'$x$')
    plt.title(title)
    
    return F

### Потенциальный барьер $V_\mathrm{max} = 40$

#### Основное состояние

При $n=0$ основное состояние имеет четную четность и, следовательно, симметрично относительно начала координат. Это означает, что начальным условием для $\psi'$ является $\psi_0' = 0$, в то время как мы можем свободно выбирать $\psi_0$, например $\psi_0 = 1$, поскольку это всего лишь вопрос нормализации. Энергия рассчитывается с использованием метода деления пополам с нижней границей $E=0$ и верхней границей $E=40$, поскольку разумно предположить, что существует состояние с энергией $E<V_\mathrm{max}$. Это приводит к следующему коду, в котором используются ранее определенные функции:

In [ ]:
b = 0.2
Vmax = 40

n = 1000                     # Number of points per unit on x-axis
dx = 1.0/n                   # Step length
p = 1                        # Which x-values to include
x = np.linspace(0, p, p*n, True)

V = Vmax/b**4*(x**2-b**2)**2 # Potential V(x)

f_0 = 1.0                    # Initial value psi_0
df_0 = 0.0                   # Initial value psi'_0

res1 = forwardShoot(V=V, dx=dx, psi_0=f_0, dpsi_0=df_0, E_u=40)
E1_0 = res1.E
print(r'The energy is: %s' % E1_0)

Полученная энергия равна $E_0=29.92$. Теперь мы нормализуем и строим волновую функцию. При нормализации мы хотим избежать включения областей, в которых волновая функция взрывается. Другими словами, мы хотим включить только значения $x$, где $\psi$ хорошо себя ведет. В этом случае путем проверки мы обнаруживаем, что $x \in [-0.7, 0.7]$.

In [ ]:
F0 = normPlot(V=V, func=res1.func, E=E1_0, x=x, x_end=0.7, title=r'Ground state, $V_\mathrm{max}=40$')

#### Первое возбужденное состояние

Из-за аргументов симметрии теперь у нас должно быть $\psi_0 = 0$, в то время как мы можем свободно выбирать $\psi_0'$, например $\psi_0' = 4$, как в приведенном ниже коде. Здесь нижняя граница энергии - это энергия основного состояния, в то время как мы выбираем верхнюю границу, равную 100.

In [ ]:
# Initial values:
f_0 = 0.0 
df_0 = 1

res2 = forwardShoot(V=V, dx=dx, psi_0=f_0, dpsi_0=df_0, E_l=E1_0, E_u=100)
E1_1 = res2.E
print('The energy is: %s' % E1_1)

Энергия первого возбужденного состояния равна $E_1=48.44$. Снова мы нормализуем и строим волновую функцию.

In [ ]:
F1 = normPlot(V=V, func=res2.func, E=E1_1, x=x, x_end=0.6, title=r'First excited state, $V_\mathrm{max}=40$',
              sym=False, legend_loc=4)

Как мы видим, существует большая разница между собственными энергиями и волновыми функциями двух (основного и первого возбужденного) состояний. Давайте теперь увеличим потенциальный барьер и посмотрим, как это повлияет на систему.

### Потенциальный барьер $V_\mathrm{max} = 400$

#### Основное состояние

Поскольку потенциальный барьер в этом случае намного выше, что приводит к волновой функции повышенной кривизны, мы выберем меньшее начальное значение $\psi_0=0.1$, в то время как $\psi_0' = 0$, как и раньше. Еще раз, мы выбираем $V_\mathrm{max}$ в качестве верхней границы энергии.

In [ ]:
Vmax = 400
V = Vmax/b**4*(x**2-b**2)**2

# Initial values
g_0 = 0.1
dg_0 = 0.0

res3 = forwardShoot(V=V, dx=dx, psi_0=g_0, dpsi_0=dg_0, E_u=400)
E2_0 = res3.E
    
print('The energy is: %s' % E2_0)

Полученная энергия равна $E_0 = 133.96$. Построение нормализованной волновой функции:

In [ ]:
G0 = normPlot(V=V, func=res3.func, E=E2_0, x=x, x_end=0.5, title=r'Ground state, $V_\mathrm{max}=400$', legend_loc=9)

#### Первое возбужденное состояние

In [ ]:
# Initial conditions
g_0 = 0.0
dg_0 = 1

res4 = forwardShoot(V=V, dx=dx, psi_0=g_0, dpsi_0=dg_0, E_l=E2_0, E_u=400)
E2_1 = res4.E

print('The energy is: %s ' % E2_1)

Полученная энергия составляет $E_0 = 134.95$ и теперь лишь немного превышает энергию основного состояния. Построение нормализованной волновой функции:

In [ ]:
G1 = normPlot(V=V, func=res4.func, E=E2_1, x=x, x_end=0.5, title=r'First excited state, $V_\mathrm{max}=400$',
              sym=False, legend_loc=4)

Теперь мы определили четыре энергии и волновые функции. Давайте построим две волновые функции, соответствующие одному и тому же потенциальному барьеру, вместе на одном графике, чтобы упростить сравнение.

In [ ]:
plt.figure()
p1, = plt.plot(x, F0, 'b')
p2, = plt.plot(x, F1, 'r')
plt.legend([p1,p2], ['Ground State','First Excited State'])
plt.plot(-x, F0, 'b', -x, -F1, 'r', [-0.5,0.5], [0,0], 'g:')
plt.ylabel(r'$\psi(x)$')
plt.xlabel(r'$x$')
plt.ylim([-2.0, 2.5])
plt.xlim([-0.5, 0.5])
plt.title(r'Wavefunctions for $V_\mathrm{max}=40$')

plt.figure()
p1, = plt.plot(x, G0, 'b')
p2, = plt.plot(x, G1, 'r')
plt.legend([p1,p2], [r'Ground State','First Excited State'])
plt.plot(-x, G0, 'b', -x, -G1, 'r', [-0.5,0.5], [0,0], 'g:')
plt.ylabel(r'$\psi(x)$')
plt.xlabel(r'$x$')
plt.ylim([-2.5, 3.2])
plt.xlim([-0.5, 0.5])
plt.title(r'Wavefunctions for $V_{max}=400$');

Мы видим, что разница между волновыми функциями для $V_\mathrm{max}=40$ велика, и у нас есть $E_0/E_1 \approx 0.6$. С другой стороны, для $V_{max}=400$ мы видим, что волновые функции почти идентичны для $x>0$, и если мы построим абсолютный квадрат этих двух функций, мы получим почти точно такую же функцию, как видно из графика ниже. У нас также есть $E_0/E_1 \approx 1$.

In [ ]:
plt.figure()
pp1, = plt.plot(x, G0**2, 'b')
p2, = plt.plot(x, G1**2, 'r')
plt.legend([p1,p2], ['Ground State','First Excited State'])
plt.plot(-x, G0**2, 'b', -x, G1**2, 'r', [-0.5,0.5], [0,0], 'g:')
plt.ylabel(r'$\psi(x)$')
plt.xlabel(r'$x$')
plt.ylim([-0.1, 5.5])
plt.xlim([-0.5, 0.5])
plt.title(r'Absolute Square of Wavefunctions for $V_\mathrm{max}=400$');

Мы также видим, что при более высоком потенциальном барьере вероятность того, что частица окажется в классически запрещенной области, резко снижается для основного состояния, как и должно быть. 
Также обратите внимание, что, поскольку энергии и абсолютные квадраты двух волновых функций настолько похожи, когда $V_\mathrm{max} = 400$, можно построить волновые функции таким образом, что частица почти наверняка находится либо в левом, либо в правом углублении, заданном $\psi_0 - \psi_1$ и $\psi_0 + \psi_1$ соответственно, как показано на следующем графике.

In [ ]:
plt.figure()
p1, = plt.plot(x, 0.5*(G0-G1)**2,'b')
p2, = plt.plot(x, 0.5*(G0+G1)**2,'r')
plt.legend([p1,p2], [r'$|\psi_0 - \psi_1|^2/2$',r'$|\psi_0 + \psi_1|^2/2$'])
plt.plot(-x, 0.5*(G0+G1)**2,'b', -x, 0.5*(G0-G1)**2, 'r', [-0.5,0.5], [0,0], 'g:')
plt.ylabel(r'$\psi(x)$')
plt.xlabel(r'$x$')
plt.ylim([-0.1, 10.0])
plt.xlim([-0.5, 0.5]);

Другими словами, по мере увеличения потенциального барьера ситуация все больше и больше напоминает ситуацию, когда частица заключена в одну из двух отдельных ям.

___

Можно задаться вопросом: насколько велика энергия второго возбужденного состояния для $V_\mathrm{max}=400$? Близка ли она к $E_0$ и $E_1$? Чтобы выяснить это, нам нужно внести небольшое изменение в функцию forwardShoot():

In [ ]:
def forwardShoot2(V, dx, psi_0, dpsi_0, E_l=0.0, E_u=400.0, acc=1e-10):
    """Takes a potential and a vector of zeroes and uses the forward shooting method to calculate the eigenenergy
    using the upper and lower bounds given.
    
    Arguments:
        V       Potential
        dx      Step length x-axis
        psi_0   Initial value of wavefunction at x=0
        dpsi_0  Initial value of first derivative of wavefunction at x=0
        E_l     Lower bound of energy
        E_u     Upper bound of energy
        acc     Accuracy when determining the energy
        
    Returns tuple with elements
        func    Wavefunction
        E       Eigenenergy
    """
    
    f = np.zeros(len(V))
    f[0] = psi_0
    df_0 = dpsi_0
    
    E1 = E_l  # Lower bound for eigenenergy
    E2 = E_u  # Upper bound for eigenenergy
    E = E1
    DeltaE = 1
    while DeltaE > acc:  
        for i, V_ in enumerate(V[0:-1]):
            if i==0:
                f[i+1]=f[i]+dx*df_0
            else:
                f[i+1] = -f[i-1]+2*f[i]*(1-dx**2*(E-V_))
            # Implementation of bisection method. If the function value is out of bounds,
            # a new value for the energy is chosen. When the difference between the upper
            # and lower bound for the energy is smaller than the given accuracy,
            # the while loop stops, yielding a result for E.
            if f[i] > 20:
                # Changes here:
                E2 = E
                E = E2 - (E2-E1)/2
                break
            elif f[i] <- 20:
                # Changes here:
                E1 = E
                E = E1 + (E2-E1)/2
                break
        DeltaE = E2-E1
    res = Results(f, E)
    return res

Мы знаем, что второе возбужденное состояние должно быть симметричным относительно начала координат. Следовательно, мы можем использовать те же начальные условия, что и для основного состояния. Мы также используем энергию первого возбужденного состояния в качестве нижней границы.

In [ ]:
g_0 = 0.1
dg_0 = 0

res5 = forwardShoot2(V=V, dx=dx, psi_0=g_0, dpsi_0=dg_0, E_l=E2_1, E_u=400)
E2_2 = res5.E
print(r'The energy is: %s' % E2_2)

Энергия составляет $E_2=349.08$, что значительно больше по сравнению с $E_1$!

In [ ]:
G2 = normPlot(V, res5.func, E2_2, x, 0.5, title=r'Second excited state, $V_\mathrm{max}=400$', sym=True, legend_loc=8)

Теперь мы видим, что волновая функция резко изменилась по сравнению с двумя более низкими состояниями. Вблизи источника она теперь больше похожа на основное состояние для $V_\mathrm{max}=40$. Это кажется разумным, учитывая, что энергия теперь сопоставима с потенциальным барьером, что также имело место для основного состояния с более низким потенциальным барьером. Частица больше не ограничена двумя ямами, так как теперь существует значительная вероятность того, что частица окажется между ними, то есть в классически запрещенной области.